# データセットの作成

In [56]:
import os
import json
import cv2
from util import CategoriesReader

In [57]:
# データ読込先のパスの設定
train_videos = []
train_annotations= []
train_data_num = 25
for index in range(train_data_num):
    train_videos.append("./train_videos_resized/train_{:02}.mp4".format(index)) # 読込先
    train_annotations.append("./train_annotations_resized/train_{:02}.json".format(index)) # 読込先
print(train_videos)
print(train_annotations)

['./train_videos_resized/train_00.mp4', './train_videos_resized/train_01.mp4', './train_videos_resized/train_02.mp4', './train_videos_resized/train_03.mp4', './train_videos_resized/train_04.mp4', './train_videos_resized/train_05.mp4', './train_videos_resized/train_06.mp4', './train_videos_resized/train_07.mp4', './train_videos_resized/train_08.mp4', './train_videos_resized/train_09.mp4', './train_videos_resized/train_10.mp4', './train_videos_resized/train_11.mp4', './train_videos_resized/train_12.mp4', './train_videos_resized/train_13.mp4', './train_videos_resized/train_14.mp4', './train_videos_resized/train_15.mp4', './train_videos_resized/train_16.mp4', './train_videos_resized/train_17.mp4', './train_videos_resized/train_18.mp4', './train_videos_resized/train_19.mp4', './train_videos_resized/train_20.mp4', './train_videos_resized/train_21.mp4', './train_videos_resized/train_22.mp4', './train_videos_resized/train_23.mp4', './train_videos_resized/train_24.mp4']
['./train_annotations_re

In [58]:
# カテゴリファイルの読み込みを行う
categories_file = "./docs/categories" # カテゴリファイル。1行ごとにカテゴリ名が記載
category_reader = CategoriesReader(categories_file)
category_dict = category_reader.get_categories_dictionary()
print(category_dict)

{0: 'Car', 1: 'Bus', 2: 'Truck', 3: 'Svehicle', 4: 'Pedestrian', 5: 'Motorbike', 6: 'Bicycle', 7: 'Train', 8: 'Signal', 9: 'Signs'}


In [59]:
# データ保存先のパスの設定
train_images_dir = "./train_images" # 画像データを保存するフォルダ
train_label_file = os.path.join(train_images_dir, "labels.txt") # 教師データラベルを保存するファイル

# データ保存先のフォルダが存在しなければ作成する
if not os.path.exists(train_images_dir):
    os.mkdir(train_images_dir)

In [60]:
ext = "jpg" 
train_labels = []

for video_path, annotation_path in zip(train_videos, train_annotations):
    ### ①学習用動画データの画像をすべて読み込み、アノテーションデータも読み込みます。
    # 学習用動画データ
    video = cv2.VideoCapture(video_path)
    
    # アノテーションデータ
    with open(annotation_path, 'r') as f:
        train_annotations_json = json.load(f) # 辞書型に変換
    
    # ②読み込んだ1シーケンス分の画像データ配列(416x416x3)は、動画ファイル名(file_name)とシーケンス番号(sequense)をキーとして、画像ファイルとそのパス（image_path）を取得します。
    sequence = 0
    while True:
        train_label_in_1seq = []
        
        # 動画から画像を1枚 読み込む
        ret, frame = video.read()
        if ret is False:
            break
            
        # 保存先の画像ファイルのパスを取得
        video_file_basename_without_ext = os.path.splitext(os.path.basename(video_path))[0]
        image_file_name = video_file_basename_without_ext + '_' + str(sequence) + '.' + ext
        image_path = os.path.join(train_images_dir, image_file_name)
        # 画像を保存する
        cv2.imwrite(image_path, frame)
        
        # Jsonファイルからシーケンス番号に関連するground_truthの情報を読み込む
        ground_truth_in_1seq = []
        for categorical_number, category in category_dict.items():
            category_values = train_annotations_json['sequence'][sequence].get(category)
            if category_values is not None:
                # 矩形を検出したら
                for rect_info in category_values:
                    ground_truth = []
                    box2d = rect_info['box2d']
                    # 矩形情報を保存する
                    ground_truth.append(categorical_number)
                    ground_truth.append(box2d[0])
                    ground_truth.append(box2d[1])
                    ground_truth.append(box2d[2])
                    ground_truth.append(box2d[3])                    
                    ground_truth_str = ",".join([str(gt) for gt in ground_truth])
                    ground_truth_in_1seq.append(ground_truth_str)
        # 1シーケンス分の教師ラベルを取得

        train_label_in_1seq.append(image_path)
        [train_label_in_1seq.append(gt1seq) for gt1seq in ground_truth_in_1seq] 
        train_label_in_1seq_str = " ".join(train_label_in_1seq)
        train_labels.append(train_label_in_1seq_str)
        sequence += 1
    video.release()
    # 全シーケンス分の教師ラベルを取得


In [71]:
# 教師データラベルを保存する。
#train_labels_str = "\n".join(train_labels)
with open(train_label_file, 'w') as f:
    #f.write(train_labels_str)
    for train_label in train_labels:
        f.write("%s\n" % train_label)
